In [1]:
%matplotlib widget

import sys  
sys.path.insert(0, '/home/cbisot/pycode/MscThesis/')
import pandas as pd
from amftrack.util import get_dates_datetime, get_dirname, get_data_info, update_plate_info, \
get_current_folders, get_folders_by_plate_id
import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
from amftrack.pipeline.functions.node_id import orient
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.experiment_class_surf import Experiment
from amftrack.pipeline.paths.directory import run_parallel, find_state, find_state_extract, directory_scratch, directory_project


In [2]:
directory = directory_project
update_plate_info(directory)

In [3]:
len(get_current_folders(directory))
all_folders = get_current_folders(directory)

In [5]:
plate = 102
folders = all_folders.loc[all_folders['Plate']==plate]
print(len(folders))
folders = all_folders.loc[all_folders['Plate']==plate*all_folders['/Analysis/skeleton_realigned_compressed.mat']]
folders

102


,folder,Plate,PrincePos,root,strain,medium,split,Temp,CrossDate,/Img/TileConfiguration.txt.registered,...,/Analysis/skeleton_masked_compressed.mat,/Analysis/skeleton_pruned_compressed.mat,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,Pbait
48,20201229_0349_Plate40,102,40,Carrot,A5,001P100N,Y,25,20201226,True,...,True,True,True,False,True,True,False,False,"29.12.2020, 03:49:",NaN
49,20210112_1143_Plate40,102,40,Carrot,A5,001P100N,Y,25,20201226,True,...,True,True,True,False,True,False,False,False,"12.01.2021, 11:43:",NaN
51,20210106_1542_Plate40,102,40,Carrot,A5,001P100N,Y,25,20201226,True,...,True,True,True,False,True,True,False,False,"06.01.2021, 15:42:",NaN
56,20201230_2333_Plate40,102,40,Carrot,A5,001P100N,Y,25,20201226,True,...,True,True,True,False,True,True,False,False,"30.12.2020, 23:33:",NaN
71,20210103_2332_Plate40,102,40,Carrot,A5,001P100N,Y,25,20201226,True,...,True,True,True,False,True,True,False,False,"03.01.2021, 23:32:",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1425,20201227_0003_Plate40,102,40,Carrot,A5,001P100N,Y,25,20201226,True,...,True,True,True,False,True,True,False,False,"27.12.2020, 00:03:",NaN
1433,20210103_0332_Plate40,102,40,Carrot,A5,001P100N,Y,25,20201226,True,...,True,True,True,False,True,True,False,False,"03.01.2021, 03:32:",NaN
1464,20210109_0350_Plate40,102,40,Carrot,A5,001P100N,Y,25,20201226,True,...,True,True,True,False,True,True,False,False,"09.01.2021, 03:50:",NaN
1467,20201230_0336_Plate40,102,40,Carrot,A5,001P100N,Y,25,20201226,True,...,True,True,True,False,True,True,False,False,"30.12.2020, 03:36:",NaN


***Create graphs***

In [5]:
num_parallel = 100
time = '30:00'
args=[directory]
run_parallel('extract_nx_graph.py',args, folders, num_parallel, time,'extract_nx')

Submitted batch job 31259
Submitted batch job 31260


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


***Extract Width***

In [16]:
num_parallel = 80
time = '5:00:00'
skip = False
args = [directory,skip]
run_parallel('extract_width.py',args,folders,num_parallel,time,'extract_width')

***Identify Nodes***

In [4]:
num_parallel = 1
time = '12:00:00'
args=[directory]
run_parallel('extract_nodes.py',args,folders,num_parallel,time,'node_id',cpus = 32)

***Hyphae extraction***

In [13]:
num_parallel = 1
time = '5:00:00'
args = [plate, begin, end, directory]
run_parallel('hyphae_extraction.py',args,pd.DataFrame(),num_parallel,time,'hyphae')

In [16]:
 0.2 * 210 * 10**14 *30 *1.8

2.268e+17